In [6]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [7]:
data_dir="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small"
file_path="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small/000/000002.mp3"
output_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/460jFinalProject/extracted_features_combined.csv'


In [8]:
# Paths to the datasets
# tracks_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/tracks.csv'
# genres_path = '/Users/dkamboj6/Recitation/460j_fma_dataset/fma_metadata/genres.csv'
tracks_path=f"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_metadata/tracks.csv"
genres_path= f"C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_metadata/genres.csv"
# Load data
tracks = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
genres = pd.read_csv(genres_path)


In [9]:
def load_genre_metadata(tracks_path):
    # Load track metadata for genre labels
    tracks_df = pd.read_csv(tracks_path, index_col=0, header=[0, 1])
    genre_labels = tracks_df['track']['genre_top']  # Adjust if the column names are different
    return genre_labels.to_dict()

# Define the path to your tracks.csv metadata file
genre_dict = load_genre_metadata(tracks_path)


In [10]:

def extract_all_features(file_path):
    try:
        # Load the audio file
        audio, sample_rate = librosa.load(file_path, sr=None)

        # Compute the Discrete Fourier Transform (DFT) features
        dft = np.abs(librosa.stft(audio))
        dft_features = np.mean(dft, axis=1)

        # Extract other features using librosa
        mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        mfccs_processed = np.mean(mfcc.T, axis=0)
        stft = np.abs(librosa.stft(audio))
        contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate)
        contrast_processed = np.mean(contrast.T, axis=0)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        chroma_processed = np.mean(chroma.T, axis=0)
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        mel_processed = np.mean(mel.T, axis=0)

        # Combine all features into one array
        combined_features = np.hstack([mfccs_processed, contrast_processed, chroma_processed, mel_processed, dft_features])

    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, error: {e}")
        return None, None

    return combined_features, file_path.split('/')[-1].split('.')[0]  # Return features and track ID


In [43]:

def process_audio_files(base_directory, genre_dict):
    features_list = []
    track_ids = []
    genres = []

    # Walk through all files in the directory structure
    for subdir, dirs, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(subdir, file)
                features, track_id = extract_all_features(file_path)
                base_name = os.path.basename(file_path)  # Gets '000002.mp3'
                track_id_str = os.path.splitext(base_name)[0]  # Splits off the '.mp3', resulting in '000002'
                track_id = int(track_id_str)
                # print(track_id)

                if features is not None and track_id in genre_dict:
                    features_list.append(features)
                    track_ids.append(track_id)
                    genres.append(genre_dict[track_id])  # Fetch the genre from the dictionary

    return features_list, track_ids, genres




In [44]:
# Process all audio files and collect features along with genres
all_features, all_track_ids, all_genres = process_audio_files(data_dir, genre_dict)

# Create a DataFrame from the extracted features, track IDs, and genres
features_df = pd.DataFrame(all_features, index=all_track_ids)
features_df['genre'] = all_genres  # Add genre as a column
features_df.index.name = 'track_id'


C:\Users\divya\AppData\Local\Temp\ipykernel_26972\3622001746.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=None)
c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\460j\460jFinalProject\.venv\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\098\098565.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\098\098567.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\098\098569.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\099\099134.mp3, error: 


c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\460j\460jFinalProject\.venv\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\108\108925.mp3, error: 
Error encountered while parsing file: C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small\133\133297.mp3, error: 


In [47]:
display(features_df)

,0,1,2,3,4,5,6,7,8,9,...,1176,1177,1178,1179,1180,1181,1182,1183,1184,genre
track_id,,,,,,,,,,,,,,,,,,,,,
2,-122.713936,117.760094,-42.334183,38.061005,-23.012323,23.920383,-11.944865,13.687291,-7.294582,4.041143,...,0.000032,0.000033,0.000039,0.000048,0.000078,0.000110,0.000089,0.000050,0.000029,Hip-Hop
5,-162.232422,131.435989,-18.426781,51.631371,-15.444844,23.860094,-7.867664,10.686234,-0.314591,12.864646,...,0.000039,0.000041,0.000060,0.000092,0.000102,0.000146,0.000153,0.000072,0.000038,Hip-Hop
10,-75.631538,154.537613,-63.849487,22.506426,2.425359,10.459981,-13.007332,9.818939,-7.640783,3.136315,...,0.000056,0.000066,0.000069,0.000083,0.000095,0.000103,0.000103,0.000072,0.000058,Pop
140,-322.997864,133.684906,15.497437,54.755615,8.718211,37.022747,-2.116411,15.265619,6.175477,-2.599205,...,0.000041,0.000035,0.000031,0.000038,0.000050,0.000052,0.000034,0.000024,0.000022,Folk
141,-253.314560,157.769379,-6.081675,26.920168,6.813843,15.842272,-12.930041,10.373278,-0.609945,-8.300909,...,0.000029,0.000025,0.000026,0.000048,0.000095,0.000081,0.000036,0.000029,0.000023,Folk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154308,-285.825165,155.782806,-7.308323,2.493736,-4.307317,1.880550,-0.723518,2.616793,7.539652,1.717390,...,0.000013,0.000016,0.000018,0.000021,0.000046,0.000094,0.000080,0.000025,0.000010,Hip-Hop
154309,-325.746490,94.565117,12.300579,-1.013848,-16.124542,4.808063,-9.805314,-7.798704,-13.323016,-5.347288,...,0.000015,0.000016,0.000021,0.000031,0.000033,0.000028,0.000019,0.000012,0.000010,Hip-Hop
154413,-264.023285,173.237167,15.978307,47.577950,-18.508392,20.177227,18.945585,15.323055,15.927598,0.358214,...,0.001181,0.001096,0.000982,0.000926,0.000939,0.001264,0.001563,0.001546,0.001409,Pop


In [48]:
# Save the DataFrame to a CSV file
features_df.to_csv(output_path)

print(f"All features with genre saved to {output_path}")


All features with genre saved to C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/460jFinalProject/extracted_features_combined.csv
